# Kappa App for nasa log view calculation (console output)

In [ ]:
from pyspark.sql import SparkSession

# Set the necessary variables
# The following links were used to determine the necessary packages to include:
# - https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html and 
# - https://github.com/OneCricketeer/docker-stacks/blob/master/hadoop-spark/spark-notebooks/kafka-sql.ipynb  

scala_version = '2.12'  
spark_version = '3.5.3'
bootstrap_servers = ['localhost:9092']
topic_name = 'kappa-topic'
window_duration = '1 minute'
sliding_duration = '1 minute'

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.9.0'
]

spark = SparkSession.builder\
   .master("local")\
   .appName("kafka-example")\
   .config("spark.jars.packages", ",".join(packages))\
   .config("SQLConf.ADAPTIVE_EXECUTION_ENABLED.key", "false")\
   .getOrCreate()

In [ ]:
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", *bootstrap_servers)\
  .option("subscribe", topic_name)\
  .option("startingOffsets", "earliest")\
  .load()
print(kafkaDf.isStreaming)    # Returns True for DataFrames that have streaming sources
kafkaDf.printSchema()

## Structure the value column

In [ ]:
from pyspark.sql.functions import col, concat, lit, split, from_unixtime
#col_schema = ["line_no INTEGER","host STRING","time TIMESTAMP","method STRING","url STRING","response INTEGER","bytes INTEGER"]
#schema_str = ",".join(col_schema)
structured_df = kafkaDf.select(
    concat(col("topic"), lit(':'), col("partition").cast("string")).alias("topic_partition"),
    col("offset"),
    col("value").cast("string"),
    col("timestamp"),
    col("timestampType"),
    split(col("value").cast("string"),",").getItem(0).alias("line_no").cast("integer"),
    split(col("value").cast("string"),",").getItem(1).alias("host").cast("string"),
    from_unixtime(split(col("value").cast("string"),",").getItem(2)).alias("time").cast("timestamp"),
    split(col("value").cast("string"),",").getItem(3).alias("method").cast("string"),
    split(col("value").cast("string"),",").getItem(4).alias("url").cast("string"),
    split(col("value").cast("string"),",").getItem(5).alias("response").cast("integer"),
    split(col("value").cast("string"),",").getItem(6).alias("bytes").cast("integer"),   
    )

In [ ]:
from pyspark.sql.functions import window,endswith

page_count_df = structured_df.filter(structured_df["method"] == "GET")\
     .filter(structured_df["url"].endswith(".html"))\
     .groupBy (window(
        col("time"),
        window_duration,
        sliding_duration
    ),
    col("url")).count() \
     .withColumnRenamed('window.start', 'window_start') \
     .withColumnRenamed('window.end', 'window_end')

query = page_count_df.writeStream\
     .format("console")\
     .outputMode("update") \
     .start()


In [ ]:
query.awaitTermination()